In [183]:
from pdp_utils import *
prob = load_problem('./Call_7_Vehicle_3.txt')
print(prob.keys())

dict_keys(['n_nodes', 'n_vehicles', 'n_calls', 'Cargo', 'TravelTime', 'FirstTravelTime', 'VesselCapacity', 'LoadingTime', 'UnloadingTime', 'VesselCargo', 'TravelCost', 'FirstTravelCost', 'PortCost'])


Pull random call from prob["Cargo"]. 

Assign call to random vehicle.

Vehicle turns up late to destination: solution is valid but infeasible.

In [184]:
import random
import functools

vehicle_carry = np.zeros(prob["n_vehicles"] + 1)
vehicle_time = np.zeros(prob["n_vehicles"] + 1)
vehicle_cost = np.zeros(prob["n_vehicles"] + 1)
vehicle_pos = np.zeros(prob["n_vehicles"] + 1)

vehicle_actions = [[-1] for i in range(int(prob["n_vehicles"]))]
active_vehicles = [True for i in range(int(prob["n_vehicles"]))]

np.random.seed(0)
n_vehicles = prob["n_vehicles"]
cargos = prob["Cargo"].copy()
max_time = np.max(cargos, axis=0)[7]

while np.size(cargos, 0) > 0:

    # Draw random cargo from unanswered pile
    to_select = np.random.choice(np.size(cargos, 0))
    current_cargo = cargos[to_select]
    cargos = np.delete(cargos, to_select, axis=0)
    
    # Draw random vehicle that has not reached end of schedule.
    v_idx = np.random.choice([i for i, x in enumerate(zip(active_vehicles, prob["VesselCargo"])) if x[0] and x[1][to_select] == 1])    
    
    #Add to vehicle time from previous location to current location, adjust for waiting.
    vehicle_time[v_idx] = max(prob["FirstTravelTime"][v_idx][int(current_cargo[0])], current_cargo[4]) if vehicle_time[v_idx] == 0 else vehicle_time[v_idx] + max(prob["TravelTime"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])], current_cargo[4])

    # If vehicle is past max time, give cargo to dummy and deactivate vehicle
    if vehicle_time[v_idx] >= max_time:
        active_vehicles[v_idx] = False
        v_idx = int(prob["n_vehicles"])
    # Add cost for driving from old cargo end to new cargo start
    # Currently adds wrong index as indexes are shifted when items are removed
    true_idx = np.where(np.all(prob["Cargo"] == current_cargo,axis=1))[0][0]
    vehicle_actions[v_idx].append(true_idx)
    vehicle_cost[v_idx] = prob["FirstTravelCost"][v_idx][int(current_cargo[0])] if vehicle_time[v_idx] == 0 else vehicle_cost[v_idx] + prob["TravelCost"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])]
    # Update vehicle position, payload.
    vehicle_pos[v_idx] = current_cargo[0]
    vehicle_carry[v_idx] += current_cargo[2]
    
    # Add time for driving from cargo start to cargo end
    vehicle_time[v_idx] = max(vehicle_time[v_idx] + prob["TravelTime"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])], current_cargo[6])
    # Add cost for driving from cargo start to cargo end
    vehicle_cost[v_idx] += prob["TravelCost"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])]
    # Remove carry weight as we are dropping off what we picked up
    vehicle_carry[v_idx] -= current_cargo[2]

solution = [[y + 1 for y in x if y >= 0] for x in vehicle_actions]
solution = functools.reduce(lambda acc, cur: acc + cur + [0], solution, [])
print(solution)    

feasiblity, c = feasibility_check(solution, prob)
print(prob["VesselCargo"])
print(feasiblity)
print(c)





[6, 7, 0, 5, 3, 0, 1, 2, 4, 0]
[[0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1. 1.]]
False
incompatible vessel and cargo
